## Results for the Annual Summer Irrigated Cropping Area (SICA) project

In [ ]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
from osgeo import gdal, ogr
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pathlib import Path
from shapely.geometry import Polygon, MultiPolygon

### Entire catchment plot

This code will produce a timeseries of irrigated area aggregated acrosss the entire Northern Murray Darling Basin

In [ ]:
directory = "/g/data/r78/cb3058/dea-notebooks/ICE_project/results/nmdb/"

In [ ]:
#list of years to help for-loop iterate through folders
x = range(1987,2019,1)
years = []
for i in x:
    nextyear = str(i + 1)[2:]
    y = str(i) + "_" + nextyear
    years.append(str(y))
# removing years that didn't work
years =  [e for e in years if e not in ('2011_12', '2012_13')]
years.sort()

folders = os.listdir(directory)
folders.sort()

area_a = []
# area_b = []
# area_c = []

def getIrrigatedArea(shp):
    irr = gpd.read_file(shp)
#     a = irr[irr.DN==80]
#     b = irr[(irr.DN==80) | (irr.DN==75)]
    a = irr[(irr.DN==80) | (irr.DN==75) | (irr.DN==70)]
    area_a.append(a.area.sum() / 10000)
#     area_b.append(b.area.sum() / 10000)
#     area_c.append(c.area.sum() / 10000)
        
#grab sum of irrigated area for each each and add to dataframe
for year, folder in zip(years, folders): 
    #progress indicator
    print("\r", "working on year: " + year, end = '')
    getIrrigatedArea(directory+folder+"/"+"nmdb_Summer"+ year + "_IrrigatedMasked.shp")

#convert years back into integers for plot
years_dt = []
for i in years:
    x = int(i[:-3])
    years_dt.append(x)
#create pandas dataframe
df = pd.DataFrame.from_dict({'year':years_dt, 'irrigated area':area_a})#, '0.75+0.8':area_b, '0.70+0.75+0.8':area_c})
df = df.set_index('year')

In [ ]:
#plot
plot_rainfall = True

if plot_rainfall == True:
    rainfall = pd.read_csv("data/mdb_rainfall.csv", index_col=0)
    ax = df.plot(colormap='jet', linestyle='--', marker='o', figsize=(16,8))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
    ax.set_xlabel('Year (Summer Nov-Mar)')
    ax.set_ylabel('Area under Irrigation (Ha)')
    ax.grid(True, linestyle='--', alpha=0.75)
    ax.axvspan(2010, 2013, alpha=0.4, color='grey')
    
    ax2 = ax.twinx()
    color='red'
    ax2.set_ylabel('summer rainfall anomalies(mm)', color=color)
    ax2.plot(rainfall, color=color, marker='o', linestyle='--', label='rainfall anomaly')
    ax2.legend(loc=2)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_ylim(bottom=-150, top=150)
    ax2.axhline(y=0, color='red')
    
    plt.tight_layout()
    
else:
    ax = df.plot(colormap='jet', linestyle='--', marker='o', figsize=(20,8))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
    ax.set_xlabel('Year (Summer Nov-Mar)')
    ax.set_ylabel('Area under Irrigation (Ha)')
    ax.grid(True, linestyle='--', alpha=0.75)
    ax.axvspan(2010, 2013, alpha=0.4, color='grey')
    ax.set_title("Northern MDB Annual Summer Irrigated Area")
    plt.tight_layout()
    plt.savefig("/g/data1a/r78/cb3058/dea-notebooks/ICE_project/results/nmdb_plots/nmdb_irrigated_ts.pdf",
               orientation='landscape')

In [ ]:
# rainfall = pd.read_csv("data/mdb_rainfall.csv", index_col=0)
# ax = rainfall.plot(figsize=(21,8))
# ax.set_ylim(bottom=-150, top=150)
# ax.axhline(0, color='black', linestyle='--', alpha=0.5)
# plt.tight_layout

### Sub-catchment plots

This script will produce a timeseries of irrigated area for each subcatchment in the directory.  This script is slow to run beacause the geopandas overlay functions are cumbersome (this is true of other libraries as well). To improve speed R-tree spatial indexing is used. This is where a bounding box is used to clip the AOI before the overlay function is run.  This workaround is only useful where the bounding box of the subcatchment is significantly smaller than the extent of the shapefile being clipped, so be wary if re-using this code elsewhere.

In [ ]:
directory = "/g/data/r78/cb3058/dea-notebooks/ICE_project/results/nmdb/"
individual_catchments_dir ="/g/data/r78/cb3058/dea-notebooks/ICE_project/data/spatial/nmdb_individual_catchments/"

In [ ]:
#generate all the filenames we need for the loops
catchments = []
for file in os.listdir(individual_catchments_dir):
    if file.endswith(".shp"):
        catchments.append(os.path.join(individual_catchments_dir, file))

x = range(1987,2019,1)
years = []
for i in x:
    nextyear = str(i + 1)[2:]
    y = str(i) + "_" + nextyear
    years.append(str(y))
# removing years that didn't work
years =  [e for e in years if e not in ('2011_12', '2012_13')]
years.sort()

years_dt = []
for i in years:
    x = int(i[:-3])
    years_dt.append(x)

folders = os.listdir(directory)
folders.sort()

In [ ]:
for catchment in catchments:
    area_a = []
    for year, folder in zip(years, folders): 
        #progress indicator
        print("\r", "working on "+ catchment[85:-4] + ": " + year, end = '')
        #get the irrigated area shapefile
        irr = gpd.read_file(directory+folder+"/"+"nmdb_Summer"+ year + "_IrrigatedMasked.shp")
        #get a subcatchment shapefile
        catch = gpd.read_file(catchment)
        geometry = catch['geometry'].iloc[0]
        if isinstance(geometry, Polygon):
            geometry = MultiPolygon([geometry])
        #clip irr to catchment area (using R-tree spatial indexing)
        sindex = irr.sindex
        possible_matches_index = list(sindex.intersection(geometry.bounds))
        possible_matches = irr.iloc[possible_matches_index]
        irr_clip = possible_matches[possible_matches.intersects(geometry)]
#         irr_clip = gpd.overlay(irr, catch, how='intersection') #works but v.v. slow
        area_a.append(irr_clip.area.sum() / 10000)
    #create pandas dataframe
    df = pd.DataFrame.from_dict({'year':years_dt, 'irr_area':area_a})
    df = df.set_index('year')
    #create plot and save
    ax = df.plot(colormap='jet', linestyle='--', marker='o', figsize=(20,8))
    ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
    ax.set_xlabel('Year (Summer Nov-Mar)')
    ax.set_ylabel('Area under Irrigation (Ha)')
    ax.grid(True, linestyle='--', alpha=0.75)
    ax.axvspan(2010, 2013, alpha=0.4, color='grey')
    ax.set_title(catchment[85:-4])
    plt.tight_layout()
    plt.savefig("/g/data1a/r78/cb3058/dea-notebooks/ICE_project/results/nmdb_plots/"+ catchment[85:-4]+".pdf",
               orientation='landscape')       

### Cumulative Irrigatable area

What am I trying to do....show all area thats undergone irrigation during a time interval...show it every 5? years.  So I'd need to union the irrigated polygons from eg 1988-1990, 1990-1995, 1995-2000, 2000-2005, 2005-2010,2010-2015, 2015-2019
For each time adjacent saved file
    find the difference between them ---> this might be best accommplished in Arc...Rather than generate more files just use coloramps to show
    
    

In [ ]:
directory = "/g/data/r78/cb3058/dea-notebooks/ICE_project/results/nmdb/"
individual_catchments_dir ="/g/data/r78/cb3058/dea-notebooks/ICE_project/data/spatial/nmdb_individual_catchments/"
AOI = "nmdb_Summer"

In [ ]:
#create list of catchment shapefiles
catchments = []
for file in os.listdir(individual_catchments_dir):
    if file.endswith(".shp"):
        catchments.append(os.path.join(individual_catchments_dir, file))
#time ranges
t1 = [str(t) for t in range(1987,1991,1)]
t2 = [str(t) for t in range(1991,1996,1)]
t3 = [str(t) for t in range(1996,2001,1)]
t4 = [str(t) for t in range(2001,2012,1)]
t4 =  [e for e in t4 if e not in ('2011_12', '2012_13')]
t5 = [str(t) for t in range(2012,2019,1)]

#list for putting results in
gdfs_t1=[]
gdfs_t2=[]
gdfs_t3=[]
gdfs_t4=[]
gdfs_t5=[]

In [ ]:
def cliptocatchment(t):
    #progress indicator
    print("\r", "working on "+ catchment[85:-4] + ": " + t, end = '')
    nextyear= str(int(t)+1)[2:]
    irr = gpd.read_file(directory + AOI + t + "_" + nextyear + "/"+ AOI + t + "_" + nextyear+"_IrrigatedMasked.shp")
    catch = gpd.read_file(catchment)
    geometry = catch['geometry'].iloc[0]
    if isinstance(geometry, Polygon):
        geometry = MultiPolygon([geometry])
    #clip irr to catchment area (using R-tree spatial indexing)
    sindex = irr.sindex
    possible_matches_index = list(sindex.intersection(geometry.bounds))
    possible_matches = irr.iloc[possible_matches_index]
    irr = possible_matches[possible_matches.intersects(geometry)]
    shapes.append(irr)

def unionyears(timelist, gdf_list):
    shapes = []
    for t in timelist:
        cliptocatchment(t)
    x = pd.concat(shapes)
    x['year_range'] = timelist[0]+ "_"+timelist[-1]
    x = x.unary_union
    gdf_list.append(x)    

In [ ]:
for catchment in catchments:
    unionyears(t1, gdfs_t1)

In [ ]:
for catchment in catchments:
    unionyears(t2, gdfs_t2)

In [ ]:
for catchment in catchments:
    unionyears(t3, gdfs_t3)

In [ ]:
for catchment in catchments:
    unionyears(t4, gdfs_t4)

In [ ]:
for catchment in catchments:
    unionyears(t5, gdfs_t5)